In [2]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import norm
from statsmodels.sandbox.regression.gmm import GMM
from statsmodels.base.model import GenericLikelihoodModel

In [3]:
data = pd.read_csv('data.csv')
print data
#print data['Plan_ID'].nunique()

      Market_ID  Plan_ID  PPO  Network Score  Satisfaction Score   Premium  \
0             1        5    0          0.860               0.800  2.497763   
1             1       16    0          0.880               0.865  2.446189   
2             1       15    1          0.910               0.875  2.795187   
3             2        2    0          0.840               0.825  2.427448   
4             2        6    0          0.890               0.875  2.430875   
5             2        3    1          0.900               0.915  2.764621   
6             3       11    0          0.840               0.830  2.401991   
7             3        1    1          0.910               0.850  2.787919   
8             3       14    0          0.920               0.850  2.489105   
9             4       13    1          0.780               0.820  2.574669   
10            4        6    0          0.890               0.875  2.471225   
11            4        9    1          0.850               0.775

Int64Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
            ...
            591, 592, 593, 594, 595, 596, 597, 598, 599, 600],
           dtype='int64', name=u'Market_ID', length=600)
      Market_ID  Inside Good Share  Outside Good Share
0             1            0.17441             0.22094
1             1            0.26550             0.22094
2             1            0.33915             0.22094
3             2            0.18765             0.21003
4             2            0.26567             0.21003
5             2            0.33665             0.21003
6             3            0.20760             0.21646
7             3            0.33332             0.21646
8             3            0.24262             0.21646
9             4            0.26121             0.20625
10            4            0.25927             0.20625
11            4            0.27327             0.20625
12            5            0.31851             0.20080
13            5            0.206

In [131]:
#first estimate using logit

class logit(GMM):
    def __init__(self, *args, **kwds):
        # set appropriate counts for moment conditions and parameters
        super(logit, self).__init__(*args, **kwds)

    def momcond(self, params):
        #unwrap stuff
        endog = self.endog 
        share = self.instrument
        exog = self.exog 
        lshare = np.log(share['Inside Good Share']) -  np.log(share['Outside Good Share']) #my market share
        lshare_fit = np.matmul(exog,params) #linear equation    
        xi = lshare_fit - lshare
        g = exog * xi[:, np.newaxis]
        return g 

    
#set up data    
y = pd.get_dummies(data['Plan_ID'])
x = data[['Market_ID','Inside Good Share']]
z =  data[['Network Score','Satisfaction Score','Premium']]
beta_init = np.full(len(z.columns),1)

#pre-processing to calculate outside good shares
group_x = x.groupby('Market_ID').sum()
group_x['Outside Good Share'] = 1 - group_x['Inside Good Share']
joined_x = pd.merge(x,group_x[['Outside Good Share']], right_index=True, left_on = 'Market_ID')

#set up model
model1 = logit(y,z,joined_x)
result1 = model1.fit(beta_init, maxiter=2, optim_method='nm', wargs=dict(centered=False))
print(result1.summary())

Optimization terminated successfully.
         Current function value: 0.000133
         Iterations: 49
         Function evaluations: 98
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 101
         Function evaluations: 181
                                                   logit Results                                                   
Dep. Variable:     [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]   Hansen J:                    1.189e-07
Model:                                                               logit   Prob (Hansen J):                   nan
Method:                                                                GMM                                         
Date:                                                     Thu, 27 Sep 2018                                         
Time:                                                             10:52:33                                         
No. Observations:         